In [ ]:
# نصب کتابخانه‌های مورد نیاز
!pip install gdown
!pip install tqdm
!pip install huggingface_hub
!pip install timm
!pip install torchmetrics



  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.6 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=8d70ef9dde3270e01b788b30dfb697d5ba8230c97ac521d339fceda4bf8a92ee
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=522bbb50cf503cc8d5dd9dc216feb80657ead8a0ab42a391ce9afda6a9fd53c6
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch

In [ ]:
import os
from huggingface_hub import hf_hub_download
import tarfile

# دانلود دیتاست
hf_hub_download(repo_id='RayanAi/inat_train_modified',
               filename="inat_train_modified.tar.gz",
               repo_type="dataset",
               local_dir=".")

# استخراج فایل tar.gz
with tarfile.open("inat_train_modified.tar.gz", "r:gz") as tar:
    tar.extractall(path=".")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


inat_train_modified.tar.gz:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

In [ ]:
from typing import Iterator
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class Node:
    def __init__(self, name):
        self.name = name
        self._count = 0
        self.children = {}
        self._entities = []

    def add_to_node(self, path, entity, level=0):
        if level >= len(path):
            self._entities.append(entity)
            return
        part = path[level]
        if part not in self.children:
            self.children[part] = Node(path[:level+1])
        self.children[part].add_to_node(path, entity, level=level+1)
        self._count += 1

    @property
    def is_leaf(self):
        return len(self._entities) > 0

    @property
    def count(self):
        if self.is_leaf:
            return len(self._entities)
        else:
            return self._count

    @property
    def entities(self):
        if self.is_leaf:
            return list((entity, self.name) for entity in self._entities)
        else:
            child_entities = []
            for child in self.children.values():
                child_entities.extend(child.entities)
        return child_entities

    def level_iterator(self, level=None):
        if level == 0:
            yield self
        elif level is None and self.is_leaf:
            yield self
        elif self.is_leaf and level != 0:
            raise Exception("Incorrect level is specified in tree.")
        else:
            if level is not None:
                level -= 1
            for child in self.children.values():
                for v in child.level_iterator(level):
                    yield v

    def print_node(self, level=0, max_level=None):
        print(' ' * (level * 4) + f"{self.name[-1]} ({self.count})")
        for node in self.children.values():
            if max_level is None or level < max_level:
                node.print_node(level + 1, max_level=max_level)
        return

class HierarchicalDataset(Dataset):
    def __init__(self, dataset_path, level=None, transform=None, verbose=True):
        self.tree = Node(("Dataset",))  # Initialize with root
        self.level = level if level is not None else 7  # Default level 7
        self.classes = set()
        self.data = []
        self.transform = transform

        index = 0
        for group_name in sorted(os.listdir(dataset_path)):
            group_dir = os.path.join(dataset_path, group_name)
            if not os.path.isdir(group_dir):
                continue
            for image_name in sorted(os.listdir(group_dir)):
                image_path = os.path.join(group_dir, image_name)
                group = tuple(group_name.split("_")[1:])  # Assuming format like 'class_name'
                if len(group) < self.level:
                    continue  # Skip if group path is shorter than required level
                group = group[:self.level]
                self.data.append({
                    "image_path": image_path,
                    "group": group,
                })
                self.tree.add_to_node(group, index)
                index += 1
                self.classes.add(group)

        self.classes = {group: idx for idx, group in enumerate(sorted(list(self.classes)))}

        if verbose:
            print(f"Dataset Length: {len(self.data)}")
            print("Hierarchical Structure (up to level 2):")
            self.tree.print_node(max_level=2)
            print(f"Number of classes: {len(self.classes)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        image = Image.open(sample["image_path"]).convert('RGB')
        group = sample["group"][:self.level]
        target = self.classes[group]
        if self.transform:
            image = self.transform(image)
        return image, target

    def get_group_iterator(self, level=None) -> Iterator[Node]:
        for group in self.tree.level_iterator(level):
            yield group


In [ ]:
from torchvision import transforms

# افزایش داده برای مجموعه آموزشی
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.ToTensor(),  # تبدیل به تنسور قبل از RandomErasing
    transforms.Normalize((0.4556, 0.4714, 0.3700), (0.2370, 0.2318, 0.2431)),
    transforms.RandomErasing(p=0.1)  # اعمال RandomErasing بعد از تبدیل به تنسور
])

# پیش‌پردازش برای مجموعه اعتبارسنجی
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4556, 0.4714, 0.3700), (0.2370, 0.2318, 0.2431))
])


In [ ]:
import torch
from torchvision import models
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
from torch.amp import GradScaler  # استفاده از torch.amp
from torchmetrics import Accuracy

def get_model(num_classes):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

# تنظیم دستگاه
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# بارگذاری مدل و دیتاست آموزشی
dataset_path = 'train'  # مسیر دیتاست
full_dataset = HierarchicalDataset(dataset_path=dataset_path, level=2, transform=train_transform)
model = get_model(num_classes=len(full_dataset.classes)).to(device)

# تنظیمات آموزش
learning_rate = 1e-4
batch_size = 256
num_epochs = 30
validation_split = 0.1
random_seed = 42

# تقسیم‌بندی به مجموعه آموزشی و اعتبارسنجی
dataset_size = len(full_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_subset = torch.utils.data.Subset(full_dataset, train_indices)

# ایجاد یک مجموعه جدید برای اعتبارسنجی بدون اعمال transforms آموزشی
val_dataset = HierarchicalDataset(dataset_path=dataset_path, level=2, transform=val_transform, verbose=False)
val_subset = torch.utils.data.Subset(val_dataset, val_indices)

# ایجاد یک مجموعه جدید بدون اعمال transforms برای محاسبه وزن‌های کلاس
train_dataset_no_transform = HierarchicalDataset(dataset_path=dataset_path, level=2, transform=None, verbose=False)
train_subset_no_transform = torch.utils.data.Subset(train_dataset_no_transform, train_indices)

# محاسبه وزن‌های کلاس
def compute_class_weights(dataset):
    class_counts = {}
    for _, label in dataset:
        class_counts[label] = class_counts.get(label, 0) + 1
    total_samples = len(dataset)
    num_classes = len(full_dataset.classes)
    class_weights = [total_samples / (num_classes * class_counts[i]) for i in range(num_classes)]
    return torch.tensor(class_weights, dtype=torch.float).to(device)

class_weights = compute_class_weights(train_subset_no_transform)

# بارگذاری داده‌ها
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

# تعریف تابع هزینه و بهینه‌ساز
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

# تنظیم‌کننده نرخ یادگیری
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(train_loader), epochs=num_epochs, anneal_strategy='linear')


Using device: cuda
Dataset Length: 99970
Hierarchical Structure (up to level 2):
Dataset (99970)
    Animalia (49112)
        Annelida (13)
        Arthropoda (29675)
        Chordata (18518)
        Cnidaria (124)
        Echinodermata (83)
        Mollusca (699)
    Fungi (1812)
        Ascomycota (396)
        Basidiomycota (1416)
    Plantae (49046)
        Bryophyta (133)
        Chlorophyta (13)
        Marchantiophyta (22)
        Rhodophyta (22)
        Tracheophyta (48856)
Number of classes: 13


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torchmetrics import Accuracy

def train_one_epoch(model, dataloader, criterion, optimizer, device, scaler, scheduler):
    model.train()
    running_loss = 0.0
    # مشخص کردن task و تعداد کلاس‌ها
    num_classes = len(full_dataset.classes)
    accuracy = Accuracy(task="multiclass", num_classes=num_classes).to(device)
    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():  # استفاده از autocast برای mixed precision
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # به‌روزرسانی نرخ یادگیری
        scheduler.step()

        # آمارگیری
        running_loss += loss.item() * inputs.size(0)
        preds = torch.argmax(outputs, dim=1)
        accuracy.update(preds, labels)

        progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy.compute().item()*100)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = accuracy.compute().item()
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    num_classes = len(full_dataset.classes)
    accuracy = Accuracy(task="multiclass", num_classes=num_classes).to(device)
    progress_bar = tqdm(dataloader, desc="Validation", leave=False)

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(outputs, dim=1)
            accuracy.update(preds, labels)

            progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy.compute().item()*100)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = accuracy.compute().item()
    return epoch_loss, epoch_acc


In [ ]:
import torch
import os
import zipfile
from torch.amp import GradScaler

# ایجاد پوشه برای چک‌پوینت‌ها
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

best_val_acc = 0.0
scaler = GradScaler()  # استفاده از GradScaler بدون پارامتر device

for epoch in range(1, num_epochs + 1):
    print(f"Epoch {epoch}/{num_epochs}")

    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device, scaler, scheduler)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # ذخیره چک‌پوینت بهترین مدل
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        checkpoint_path = os.path.join(checkpoint_dir, 'best_checkpoint.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
        }, checkpoint_path)
        print(f"New best model saved with Val Acc: {best_val_acc:.2f}%")

    print("-" * 30)

# بارگذاری بهترین مدل
best_checkpoint = torch.load(os.path.join(checkpoint_dir, 'best_checkpoint.pth'), map_location=device)
model.load_state_dict(best_checkpoint['model_state_dict'])

# ذخیره وزن‌های نهایی مدل
torch.save(model.state_dict(), 'resnet.pth')

# ایجاد فایل ZIP برای ارسال
with zipfile.ZipFile('submission.zip', 'w') as zipf:
    zipf.write('resnet.pth')

print("وزن‌های مدل به نام 'resnet.pth' ذخیره شده و در 'submission.zip' بسته‌بندی شدند.")


Epoch 1/30


Training:   0%|          | 0/352 [00:00<?, ?it/s]<ipython-input-61-2e7b193a5f0b>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # استفاده از autocast برای mixed precision
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Validation:   0%|          | 0/40 [00:00<?, ?it/s]<ipython-input-61-2e7b193a5f0b>:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.aut

Train Loss: 2.3474 | Train Acc: 0.36%
Val Loss: 1.6850 | Val Acc: 0.67%
New best model saved with Val Acc: 0.67%
------------------------------
Epoch 2/30


Train Loss: 1.7315 | Train Acc: 0.59%
Val Loss: 1.1817 | Val Acc: 0.76%
New best model saved with Val Acc: 0.76%
------------------------------
Epoch 3/30


Train Loss: 1.4352 | Train Acc: 0.66%
Val Loss: 1.0128 | Val Acc: 0.80%
New best model saved with Val Acc: 0.80%
------------------------------
Epoch 4/30


...

Train Loss: 0.4696 | Train Acc: 0.78%
Val Loss: 1.3866 | Val Acc: 0.86%
------------------------------
Epoch 29/30


Train Loss: 0.4723 | Train Acc: 0.79%
Val Loss: 1.3022 | Val Acc: 0.86%
New best model saved with Val Acc: 0.86%
------------------------------
Epoch 30/30


Train Loss: 0.4305 | Train Acc: 0.79%
Val Loss: 1.3121 | Val Acc: 0.87%
New best model saved with Val Acc: 0.87%
------------------------------


<ipython-input-62-1a613a92e49f>:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_checkpoint = torch.load(os.path.join(checkpoint_dir, 'best_checkpoint.pth'), map_locat

وزن‌های مدل به نام 'resnet.pth' ذخیره شده و در 'submission.zip' بسته‌بندی شدند.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# مسیر مدل اصلی شما
model_path = '/content/resnet.pth'

# بارگذاری مدل
model = torch.load(model_path)

# تبدیل پارامترهای مدل به نیمه‌دقت
model.half()

# ذخیره مدل با دقت نیمه
torch.save(model, '/content/resnet_fp16.pth')


<ipython-input-54-f7e34b0ed487>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


AttributeError: 'collections.OrderedDict' object has no attribute 'half'

In [ ]:
import zipfile
import os

def zip_multiple_files(input_paths, output_zip_path, compression_level=5):
    # Ensure compression level is within 1-9
    compression_level = max(1, min(compression_level, 1))

    # Create a zip file with the specified compression level
    compression = zipfile.ZIP_DEFLATED

    # Create the zip file
    with zipfile.ZipFile(output_zip_path, 'w', compression) as zipf:
        for input_path in input_paths:
            # Check if the file or directory exists
            if not os.path.exists(input_path):
                print(f"{input_path} does not exist.")
                continue

            # If it's a directory, recursively add files
            if os.path.isdir(input_path):
                for root, dirs, files in os.walk(input_path):
                    for file in files:
                        file_full_path = os.path.join(root, file)
                        zipf.write(file_full_path,
                                   os.path.relpath(file_full_path,
                                                   os.path.join(input_path, '..')))
            # If it's a single file, add it to the zip file
            else:
                zipf.write(input_path, os.path.basename(input_path))

    print(f"Successfully zipped files to {output_zip_path} with compression level {compression_level}")

# Example usage:
input_paths = ['/content/model.py', '/content/model.pth']  # List of files or directories to zip
output_zip_path = '/content/submission1.zip'  # Path to save the output zip file
compression_level = 5  # Compression level from 1 (fastest) to 9 (most compressed)
zip_multiple_files(input_paths, output_zip_path, compression_level)


In [ ]:
!rm -rf __pycache__

In [ ]:
!rm -rf model.py /content/resnet.pth /content/submission2.zip /content/checkpoints

In [ ]:
!mv /content/submission.zip /content/submission5.zip

In [ ]:
!cp /content/submission5.zip /content/drive/MyDrive/ML/Rayan/Q3/submission